
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.1/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import copy

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader,build_detection_train_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import transforms as T
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [6]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/MyDrive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [7]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  train_dirs = '{}/data/train'.format(BASE_DIR)
  test_dirs = '{}/data/test'.format(BASE_DIR)
  dataset = []
  json_file = os.path.join(data_dirs, "train.json")
  with open(json_file) as f:
    imgs_anns = json.load(f)

  dataset_dicts = []
  objs = []
  name_change = ""
  if set_name == "test":             
    for filenames in os.listdir(test_dirs):
      record = {}  
      if filenames.endswith(".png"):
        filename = os.path.join(test_dirs, filenames)
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["height"] = height
        record["width"] = width
        obj = {}
        record["annotations"] =objs
        dataset.append(record) 
  elif set_name == "train" or set_name == "val":
    for i in range(len(imgs_anns)):
      v = imgs_anns[i]
      record = {}

      filename = os.path.join(train_dirs, v["file_name"])
      img = Image.open(filename)
      width, height = img.size

      if name_change!=filename:
        record["file_name"] = filename
        record["image_id"] = v["image_id"]
        record["height"] = height
        record["width"] = width

        dataset.append(record)
        objs = []
        name_change = filename

      obj = {
                "bbox": v["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": v["segmentation"],
                "category_id": 0,
        }
      objs.append(obj)
      record["annotations"] = objs

  return dataset

In [8]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
# DatasetCatalog.list()
# DatasetCatalog.remove('data_detection_train')
# DatasetCatalog.remove('data_detection_test')
# DatasetCatalog.remove('data_detection_val')
for d in ["train", "val", "test"]:
  DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
  MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_detection_train")

# DatasetCatalog.list()

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata = plane_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [11]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_test",)

cfg.DATALOADER.NUM_WORKERS = 4
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005 # 0.00025 -> 0.0005 -> 0.001 -> 0.0005(final)
cfg.SOLVER.MAX_ITER = 5000  # 500 -> 1000 -> 5000 (final)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  


### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=True)   #default False
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [13]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):  
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.3, 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir= "./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

### Data Loader

In [16]:
from torch.jit import Error
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
def get_instance_sample(data, idx, img=None):
  img = Image.open(data["file_name"])
  plane = data["annotations"][idx]

  img_seg = plane["segmentation"]
  img_bbox = plane["bbox"]
  x = int(img_bbox[0])
  y = int(img_bbox[1])
  width = int(img_bbox[2])
  height = int(img_bbox[3])

  # get the object image
  im1 = img.crop((img_bbox[0], img_bbox[1], img_bbox[0]+width, img_bbox[1]+height))
  obj_img = im1.resize((128,128),Image.ANTIALIAS)
  obj_img = np.resize(obj_img,(128,128,3)) 

  # get the object mask
  new_seg = []
  for i in img_seg:
    new_row = []
    for j in range(0,len(i)-1,2):
      i[j] = i[j]-img_bbox[0]
      i[j+1] = i[j+1]-img_bbox[1]
      new_row.append(i[j])
      new_row.append(i[j+1])
    new_seg.append(new_row)        
  obj_mask = detectron2.utils.visualizer.GenericMask(new_seg,height,width).mask
  obj_mask = cv2.resize(obj_mask, (128, 128))
    
  return obj_img, obj_mask

In [17]:
def get_instance_sample_test(data, idx, prepared_imageset):
  for i in prepared_imageset:
    if i["file_name"] == data["file_name"]:
      obj_img = i["annotations"][idx]["obj_img"]
      obj_mask = i["annotations"][idx]["obj_mask"]
      break
  return obj_img, obj_mask

In [18]:

'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''
class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])
  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    ann_index = idx[1]
    
    if self.set_name == "train":
      img, mask = get_instance_sample(data, ann_index)
    else:
      img, mask = get_instance_sample_test(data, ann_index, self.prepared_imageset)

    img, mask = self.numpy_to_tensor(img, mask)
    img = img.reshape((3,128,128))
    mask = mask.reshape((1,128,128))

    return img, mask

  def set_prepared_imageset(self, prepared_imageset):
    self.prepared_imageset = prepared_imageset

def get_plane_dataset(set_name, batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [19]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )
    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )
    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        self.input_conv = conv(3, 8)
        self.down_1 = down(8, 16)
        self.down_2 = down(16, 32)
        self.down_3 = down(32, 64)
        self.down_4 = down(64, 128)
        self.down_5 = down(128, 256)
        self.down_6 = down(256, 512)
        
        # Decoder  
        self.up_1 = up(512, 256)
        self.up_2 = up(256, 128)
        self.up_3 = up(128, 64)
        self.up_4 = up(64, 32)
        self.up_5 = up(32, 16)
        self.up_6 = up(16, 8)
        self.up_7 = conv(8, 3)
        self.output_conv = conv(3, 1, False)
        
        self.normal_1 = nn.BatchNorm2d(8)
        self.normal_2 = nn.BatchNorm2d(16)
        self.normal_3 = nn.BatchNorm2d(32)
        self.normal_4 = nn.BatchNorm2d(64)
        self.normal_5 = nn.BatchNorm2d(128)
        self.normal_6 = nn.BatchNorm2d(256)

    def forward(self, input):
      y_1 = self.input_conv(input)
      y_2 = self.down_1(y_1)
      y_3 = self.down_2(y_2)
      y_4 = self.down_3(y_3)
      y_5 = self.down_4(y_4)
      y_6 = self.down_5(y_5)
      y_7 = self.down_6(y_6)
      y_7 = self.normal_6(F.relu(self.up_1(y_7) + y_6))
      y_6 = self.normal_5(F.relu(self.up_2(y_7) + y_5))
      y_5 = self.normal_4(F.relu(self.up_3(y_6) + y_4))
      y_4 = self.normal_3(F.relu(self.up_4(y_5) + y_3))
      y_3 = self.normal_2(F.relu(self.up_5(y_4) + y_2))
      y_2 = self.normal_1(F.relu(self.up_6(y_3) + y_1))
      y = self.up_7(y_2)
      output = self.output_conv(y)
      return output

### Training

In [20]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''
import warnings

warnings.filterwarnings('ignore')

# Set the hyperparameters
num_epochs = 50       # 5 -> 25 -> 50
batch_size = 8
learning_rate = 0.05    # 0.01 -> 0.05
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

img_with_mask_list = []
for (img, mask) in tqdm(loader):
  img_with_mask_list.append((img,mask))

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(img_with_mask_list):
    # img = img.clone().detach().requires_grad_(True)
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    # mask = mask.clone().detach().requires_grad_(True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

Epoch: 0, Loss: 0.29379168152809143


  0%|          | 0/998 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.21450768411159515


...

Epoch: 47, Loss: 0.0537024550139904


  0%|          | 0/998 [00:00<?, ?it/s]

Epoch: 48, Loss: 0.05308536812663078


  0%|          | 0/998 [00:00<?, ?it/s]

Epoch: 49, Loss: 0.05262439697980881


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() 
loader, dataset = get_plane_dataset('train', batch_size)

count = 0
overall_score = 0
datalist = []
for (img, mask) in tqdm(img_with_mask_list):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    pred = model(img)
    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    for batch in range(pred.shape[0]):
      obj_img = img[batch].cpu().numpy()
      obj_img = np.resize(obj_img,(128,128))

      pred_mask = pred[batch].cpu()
      pred_mask = pred_mask.sigmoid().numpy().squeeze(axis = 0)

      for i in pred_mask: # can change threshold 0.25
        i[i < 0.25] = 0
        i[i >= 0.25] = 1

      true_mask = mask[batch].cpu().numpy()
      true_mask = np.resize(true_mask,(128,128))

      datalist.append([obj_img,true_mask,pred_mask])

      intersect = np.logical_and(pred_mask,true_mask)
      union = np.logical_or(pred_mask,true_mask)
      IoUscore = np.sum(intersect)/np.sum(union)    
      overall_score += IoUscore
      count += 1

print("IoU:",overall_score/count)
print("\n #images: {}, Mean IoU: {}".format(_, _))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
for i in np.random.randint(len(datalist), size=3):  
  plt.imshow(datalist[i][0], cmap='gray') # original image
  plt.figure()
  plt.imshow(datalist[i][1], cmap='gray') # true mask
  plt.figure()
  plt.imshow(datalist[i][2], cmap='gray') # prediction mask
  plt.figure()

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [24]:
def get_prediction_dataset(set_name, data_list,prepared_imageset,batch_size=2):
    dataset = PlaneDataset(set_name, data_list)
    dataset.set_prepared_imageset(prepared_imageset)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=False)
    return loader, dataset

In [25]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''
def get_prediction_mask(data, imageset):
  ori_image = Image.open(data["file_name"])
  filename = data["file_name"]
  sample = []
  if len(data["annotations"])!=0:
    sample.append(data)
    instance_sample = data
  else:
    for image in imageset:
      if image["file_name"] == data["file_name"]:
        sample.append(image)
        for instance in image:
          instance_sample = image  
  width = data["width"]
  height = data["height"]

  if len(data["annotations"])!=0:
    planes = []
    for i in data["annotations"]:
      planes.append(i["segmentation"][0])
    gt_mask = detectron2.utils.visualizer.GenericMask(planes,height,width).mask
  else:
    gt_mask = None

  pred_masks = []
  batch_size = 8
  model = MyModel().cuda()
  model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
  model = model.eval() 
  loader, dataset = get_prediction_dataset("prediction", sample, imageset, batch_size)

  for img, _ in tqdm(loader):
    with torch.no_grad():
      img = img.cuda()
      pred = model(img)
      batch_size = pred.shape
    for batch in range(batch_size[0]):
      pred_mk = pred[batch].cpu()
      pred_mk = pred_mk.sigmoid().numpy()
      pred_mk = np.squeeze(pred_mk,0)
      for i in pred_mk:
        i[i<0.25] = 0
        i[i>= 0.25] = 1
      pred_masks.append(pred_mk)
    for s in imageset:
      if s["file_name"] == data["file_name"]:
        for i in range(len(pred_masks)):
          s["annotations"][i]["obj_mask"] = pred_masks[i]
  gt = np.zeros((height, width))

  for i in range(len(pred_masks)):
    row_count = 0
    sample_width = int(instance_sample["annotations"][i]["bbox"][3])
    sample_height = int(instance_sample["annotations"][i]["bbox"][2])
    sample_x = int(instance_sample["annotations"][i]["bbox"][0])
    sample_y = int(instance_sample["annotations"][i]["bbox"][1])
    pred_masks[i]= cv2.resize(pred_masks[i],(sample_height,sample_width))
 
    for y in pred_masks[i]: 
      count=0 
      for x in y:
        if(x>0.5):
          gt[row_count+sample_y,count+sample_x] = i+1
        count+=1   
      row_count+=1
 
  if len(data["annotations"])!=0:
    gt_mask = torch.from_numpy(gt_mask)
  else:
    gt_mask = gt
  gt = torch.from_numpy(gt)
  gt = gt.cuda()
  return ori_image, gt_mask, gt

### Visualization and Submission

In [26]:
image_croppingset = get_detection_data("train")
train_crop = []
for v in image_croppingset:
  record = {}
  record["file_name"] = v["file_name"]
  img = Image.open(v["file_name"])
  objs = []
  for i in v["annotations"]:
    img_seg = i["segmentation"]
    img_bbox = i["bbox"]
    height = int(img_bbox[2])
    width = int(img_bbox[3])
    new_seg = []
  
    for i in img_seg:
      new_row = []
      for j in range(0,len(i)-1,2):
        i[j] = i[j]-img_bbox[0]
        i[j+1] = i[j+1]-img_bbox[1]
        new_row.append(i[j])
        new_row.append(i[j+1])
      new_seg.append(new_row)        
    im1 = img.crop((img_bbox[0], img_bbox[1], img_bbox[0]+height, img_bbox[1]+width))
    obj_img = im1.resize((128,128),Image.ANTIALIAS)
    obj_img = np.resize(obj_img,(128,128,3)) 
    obj_mask = detectron2.utils.visualizer.GenericMask(new_seg,width,height).mask
    obj_mask = cv2.resize(obj_mask, (128, 128))
    obj = {
        "obj_img":obj_img,
        "obj_mask":obj_mask,
    }
   
    objs.append(obj)     
  record["annotations"]=objs
  train_crop.append(record)

In [27]:
test_dicts = get_detection_data("test")
test_crop = []
for pic in test_dicts:
  record = {}
  record["file_name"] = pic["file_name"]

  objs = []

  img = cv2.imread(pic["file_name"])
  outputs = predictor(img)
  pil_img = Image.open(pic["file_name"])
  for i in range(len(outputs["instances"])):
    one_plane = outputs["instances"][i].pred_boxes
    one_plane = one_plane.tensor[0].cpu().numpy()
    height = (one_plane[2])
    width = (one_plane[3])
    im1 = pil_img.crop((one_plane[0], one_plane[1], height, width))
    obj_img = im1.resize((128,128),Image.ANTIALIAS)
    obj_mask = np.zeros((128,128))
    
    obj_img = np.resize(obj_img,(128,128,3))
    obj = {
        "bbox":one_plane,
        "obj_img":obj_img,
        "obj_mask":obj_mask,
    }  
    objs.append(obj)

  record["annotations"]=objs
  test_crop.append(record) 

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
dataset = get_detection_data('train')
for i in np.random.randint(0,50,3):
  img, true_mask, pred_mask = get_prediction_mask(dataset[i], train_crop)
  pred_mask = pred_mask.cpu().numpy()
  true_mask = true_mask.cpu().numpy()

  plt.imshow(img)
  plt.figure()
  plt.imshow(true_mask, cmap='gray')
  plt.figure()
  plt.imshow(pred_mask, cmap='gray')
  plt.figure()
  


In [30]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''
preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, train_crop)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

In [ ]:
'''
# Writing the predictions of the test set
'''
# my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]

  img, true_mask, pred_mask = get_prediction_mask(sample,test_crop)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()

## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_test",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005 
cfg.SOLVER.MAX_ITER = 5000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

### Network

### Training

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)   #default False

trainer.train()

### Evaluation and Visualization

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir= "./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

In [ ]:
dataset_dicts = get_detection_data("test")
count = 0
for d in random.sample(dataset_dicts, 3):  
    count +=1
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.3, 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    cv2.imwrite("part4predict{}.png".format(count), out.get_image()[:, :, ::-1])
   